In [22]:
from glob import glob
import os
import numpy as np
import matplotlib.pyplot as plt
import shutil
from torchvision import transforms
from torchvision import models
import torch
from torch.autograd import Variable
import torch.nn as nn
from torch.optim import lr_scheduler
from torch import optim
from torchvision.datasets import ImageFolder
from torchvision.utils import make_grid
import time
%matplotlib inline
device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)

cpu


# Dataset 정리

In [3]:
print(os.getcwd())

C:\Users\User\deep_learning_pytorch


In [5]:
transformation = transforms.Compose([transforms.Resize((224,224)),transforms.ToTensor(),transforms.Normalize([0.485, 0.456, 0.406], [0.229,0.224, 0.225])])
train = ImageFolder('dog_cat/dogs-vs-cats/train/train',transformation)
valid = ImageFolder('dog_cat/dogs-vs-cats/train/valid',transformation)

train_dataloader = torch.utils.data.DataLoader(train,batch_size=64)
valid_dataloader = torch.utils.data.DataLoader(valid,batch_size=64)

In [8]:
dataset_sizes = {'train':len(train_dataloader.dataset), 'valid':len(valid_dataloader.dataset)}
dataloaders = {'train': train_dataloader, 'valid': valid_dataloader}

# Model 설정

In [9]:
from torchvision import models
vgg = models.vgg16(pretrained=True)

Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to C:\Users\User/.cache\torch\hub\checkpoints\vgg16-397923af.pth


  0%|          | 0.00/528M [00:00<?, ?B/s]

In [11]:
#features parameter 고정
for param in vgg.features.parameters():
    param.requires_grad = False

In [18]:
vgg.classifier[6].out_features = 2

In [24]:
model = vgg
print(model)

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

In [17]:
import torch.optim as optim
loss_fn = torch.nn.CrossEntropyLoss()
optimizer = optim.SGD(vgg.classifier.parameters(),lr = 0.0001, momentum = 0.5)

# Train, Test 함수설정

In [29]:
def train(dataloader, model, loss_fn, optimizer):
    model.train()
    size = len(dataloader.dataset)
    for batch, (X,y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)
        
        pred = model(X)
        loss = loss_fn(pred,y)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if batch %100 == 0:
            loss, current = loss.item(), batch*len(X)
            print(f"loss:{loss:>7f}[{current:>5d}/{size:>5d}]")

In [26]:
def test(dataloader, model, loss_fn):
    model.eval()
    size = len(datalodaer.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0,0
    with torch.no_grad():
        for X,y in enumerate(dataloader):
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred,y).item()
            correct += (pred.argmax(1)==y).type(torch.float).sum().item
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Acuuracy: {(100*correct):>0.1f}%,Avg loss:{test_loss:>8f} \n")

# Training

In [30]:
epochs = 20
for t in range(epochs):
    print(f"Epoch {t+1}\n")
    train(train_dataloader, model, loss_fn, optimizer)
    test(valid_dataloader, model, loss_fn)
print("Done!")

Epoch 1

loss:16.966499[    0/23000]


KeyboardInterrupt: 

In [16]:
torch.save(model.state_dict(), "model.pth")
print("Saved Pytorch model state to model.pth")

[]
